In [64]:
import time
import random
from datetime import datetime, date, timedelta

# Hubbub Reservations

We love Hubbub! and testing!!

## Introduction

This Jupyter Notebook is a walkthrough of how to use this testing script for Hubbub reservations. A Hubbub reservation has been simplified here for the purpose of testing new features on date manipulation without all of the other attributes which make reservations complex.

Date manipulation is at the core of the Hubbub Shop infrastructure, so it is incredibly important to make sure that *any new feature is thoroughly tested with >10000 simulations*. This notebook will hopefully make it easier to:

1. see previous tests
2. re-run old tests
3. debug undiscovered problems
4. and create new features!

This is a living document, so once your feature and test have been successfully built, document it here for posterity.

## Get Started

Begin by importing the modules at the top of the notebook. You'll need all of them to make the tests work. You will also need to execute the input box below. These functions and classes define the 'dummy' objects which mock the real Hubbub reservations. They are only for testing date manipulation.

In [48]:
# Constants to be referenced throughout the notebook.
LOWER_LIMIT = date(2020, 1, 1)
UPPER_LIMIT = date(2025, 1, 1)
DATE_FORMAT = '%Y-%m-%d'


# Date generator is used to produce a random date within the range
# defined by the constants LOWER_LIMIT and UPPER_LIMIT.
def date_generator(start=LOWER_LIMIT, end=UPPER_LIMIT):
    start_date = start
    end_date = end
    time_between_dates = end_date - start_date
    days_between_dates = time_between_dates.days
    random_number_of_days = random.randrange(days_between_dates)
    return start_date + timedelta(days=random_number_of_days)

# Returns a tuple of random dates, where start < end
def create_date_range():
    start = date_generator()
    end = date_generator()
    
    #cannot have intervals like [10, 10]... 
    #could create overlapping reservations
    while end <= start:
        start, end = create_date_range()
    return start, end

# This is the mock class for the Hubbub reservation. It uses the same variable
# names as the original as of the time writing this, so it is easily portable.
class Reservations:
    
    # If the user does not define the start and end, then they are randomly generated
    def __init__(self, min_start=None, max_end=None):
        if min_start and max_end:
            self.date_started = min_start
            self.date_ended = max_end
        else:
            self.date_started, self.date_ended = create_date_range()
            
        # NOTE: self._length does NOT exist in the Hubbub reservations, this is just for testing
        self._length = (self.date_ended - self.date_started).days
    
    # Representation of the reservation
    def __repr__(self):
        return f"<{self.date_started.strftime(DATE_FORMAT)} to {self.date_ended.strftime(DATE_FORMAT)}>"
    
class Calendars(Reservations):
    
    def __init__(self, min_start=None, max_end=None):
        super().__init__(min_start, max_end)
        self.reservations = []
    
    def add(self, reservation):
        self.reservations.append(reservation)
        
    def remove(self, reservation):
        _reservations = self.reservations.copy()
        i = 0
        while i < len(_reservations):
            if res.date_started == reservation.date_started:
                if res.date_ended == reservation.date_ended:
                    _reservations.remove(res)
                    break
            i += 1
        self.reservations = _reservations

Once you've run the input above, you are ready to go! From here, this will just cover the tests done on reservations to date. Add you tests to the end of the file and good luck! Include thorough notes explaining what you did, what the results are, and unexpected challenges.

## Tests

### Scheduler

Scheduler is a function which accepts an object of type Reservations and an object of type Calendars, and returns either **True**, **False**, or **None**.

1. True - returned when the input reservation does not conflict with existing reservations in calendar. Does not edit the calendar.
2. False - returned when the input does conflict with a reservation in calendar.
3. None - returned when the calendar is no longer schedulable.

There are two implementations of the scheduler() below: a) an iterative implementation, and b) a recursive implementation. Both should work just fine; however the recursive is currently deployed.

In [54]:
# In Hubbub, this function is built into calendar as an instancemethod however, we separate it here for testing.

# Iterative scheduler (id = 0)
def iterative_scheduler(calendar, reservation):
    bookings = calendar.reservations.copy()
    if len(bookings) == 0:
        if date.today() < calendar.date_ended:
            return True
        else:
            return None
    else:
        bookings.sort(key = lambda res: res.date_ended)
        for i in range(len(bookings)):
            if bookings[i].date_ended <= reservation.date_started:
                if i + 1 < len(bookings):
                    if bookings[i + 1].date_started >= reservation.date_ended:
                        return True
                elif i + 1 == len(bookings):
                    return True
        if bookings[0].date_started >= reservation.date_ended:
            return True
        return False
    
# Recursive scheduler (id = 1) - in production
def recursive_scheduler(bookings, bounds, reservation):
    bookings.sort(key = lambda res: res.date_ended)
    _bookings = bookings.copy()
    if len(_bookings) == 0:
        if date.today() < bounds.date_ended:
            return True
        else:
            return None
    else:
        comparison_res = _bookings.pop(0)
        if comparison_res.date_ended <= reservation.date_started:
            return recursive_scheduler(_bookings, bounds, reservation)
        elif comparison_res.date_started >= reservation.date_ended:
            return recursive_scheduler(_bookings, bounds, reservation)
        else:
            return False


Now to write the test function. The test function will be different depending on which scheduler you choose, but it works fundamentally the same way.

In [65]:
# Takes the number of simulations you want to run and returns an error if a reservation is incorrectly added.
# return f"Calendar is valid. See for yourself: {bookings} \n\nand this is the empty array {_bookings}"
def test_scheduler(function_id=None, simulations=100, debug=False):
    start_time = time.time()
    calendar = Calendars(min_start=LOWER_LIMIT, max_end=UPPER_LIMIT)
    if debug:
        print("The calendar is initialized.")
        print("The calendar bounds: ", calendar)
        print("The calendar reservations: ", calendar.reservations)
    i = 0
    if function_id == 1:  
        while i < simulations:
            test_reservation = Reservations()
            if recursive_scheduler(calendar.reservations, calendar, test_reservation):
                calendar.add(test_reservation)
                if debug:
                    print(f"Your reservation for {test_reservation} is valid, it seems.")
                    print(" Updated calendar reservations: ", calendar.reservations)
            elif debug:
                print("It's invalid, sorry.")
                print("Your request: ", test_reservation)
                print("Existing reservations: ", calendar.reservations)
            i += 1
    elif function_id == 0:
        while i < simulations:
            test_reservation = Reservations()
            if iterative_scheduler(calendar, test_reservation):
                calendar.add(test_reservation)
                if debug:
                    print(f"Your reservation for {test_reservation} is valid, it seems.")
                    print(" Updated calendar reservations: ", calendar.reservations)
            elif debug:
                print("It's invalid, sorry.")
                print("Your request: ", test_reservation)
                print("Existing reservations: ", calendar.reservations)
            i += 1
    else:
        raise Exception("please select a scheduler to test!")
    calendar.reservations.sort(key = lambda res: res.date_ended)
    test_bookings = calendar.reservations.copy()
    while len(test_bookings) > 1:
        if test_bookings[0].date_ended <= test_bookings[1].date_started:
            valid_reservations = test_bookings.pop(0)
        else:
            return f"Error, the comparison between {test_bookings[0]} and {test_bookings[1]} failed."
    return f"Test passed. {simulations} simulations executed in {time.time() - start_time} seconds."

Now it's time to tests! Run the code below to simulate scheduler. Make sure you select a version of scheduler to test. ID = 0 is for iterative while ID = 1 is for recursive.

In [69]:
# Test conditions
ID = 1
SIMULATION_COUNT = 100
DEBUG = False

# test_scheduler(function_id=ID, simulations=SIMULATION_COUNT, debug=DEBUG)

And that wraps it up! Leave a note or email at [Ade Balogun](mailto:adb2189@columbia.edu) if you want to suggest improvements to this test, or believe it is failing in some way.

### Availabilities

These functions are important for determining if an item is available to on a select date (check_availability) and returning the nearest range of dates for which it is available (next_availability).

In [63]:
# Checks to see whether or not the calendar has a reservation which overlaps with the input date.
def check_availability(calendar, check_date):
    if len(calendar.reservations) == 0:
        return True
    for reservation in calendar.reservations:
        if reservation.date_started <= check_date:
            if reservation.date_ended >= check_date:
                return False
    return True

Now for the test function for check_availability:

In [83]:
# Test check_availability
def test_check_availability(simulations=100, debug=False):
    start_time = time.time()
    i = 0
    while i < simulations:
        toggle = random.choice([True, False])
        
        test_date = date_generator()
        calendar = Calendars(min_start=LOWER_LIMIT, max_end=UPPER_LIMIT)
        
        if toggle:
            test_start = test_date - timedelta(days=2)
            test_end = test_date + timedelta(days=2)
            test_reservation = Reservations(min_start=test_start, max_end=test_end)
            if debug:
                print("Test date has been initialized: ", test_date)
                print("Test reservation has been initialized to purposefully block the test_date: ", test_reservation)
            calendar.add(test_reservation)
        elif debug:
            print("The test date should be valid.")
        result = check_availability(calendar, test_date)
        if toggle and result:
            raise Exception(f"Test failed! date: {test_date} is NOT in range {calendar}")
        elif toggle == False and result == False:
            raise Exception(f"Test failed! date: {test_date} IS in range {calendar}")
        i += 1
    return f"Test passed. {simulations} simulations executed in {time.time() - start_time} seconds."

Now execute the code below to run the test.

In [87]:
# Test conditions
SIMULATION_COUNT = 100
DEBUG = True

# test_check_availability(simulations=SIMULATION_COUNT, debug=DEBUG)

Now to test next_availability:

In [93]:
# A function which returns the next available date range in the calendar
def next_availability(calendar, MARGIN=2):
    closest_operating_date = date.today() + timedelta(days=MARGIN)
    calendar.reservations.sort(key = lambda res: res.date_ended)
    _bookings = calendar.reservations.copy()
    if len(_bookings) == 0:
        if closest_operating_date > calendar.date_started:
            return [closest_operating_date, calendar.date_ended]
        else:
            return [calendar.date_started, calendar.date_ended]
    for i in range(len(_bookings)):
        if i + 1 < len(_bookings):
            if _bookings[i].date_ended < _bookings[i + 1].date_started:
                if closest_operating_date < _bookings[i].date_ended:
                    return [_bookings[i].date_ended, _bookings[i + 1].date_started]
                elif closest_operating_date < _bookings[i + 1].date_started:
                    return [closest_operating_date, _bookings[i + 1].date_started]
        elif _bookings[0].date_started > closest_operating_date:
            if closest_operating_date >= calendar.date_started:
                return [closest_operating_date, _bookings[0].date_started]
            elif _bookings[0].date_started > calendar.date_started:
                return [calendar.date_started, _bookings[0].date_started]
            else:
                return [_bookings[i].date_ended, calendar.date_ended]
        elif closest_operating_date < calendar.date_ended:
            if closest_operating_date > _bookings[i].date_ended:
                return [closest_operating_date, calendar.date_ended]
            else:
                return [_bookings[i].date_ended, calendar.date_ended]
        else:
            return [closest_operating_date, closest_operating_date]

The test below most likely works, but if the logic does not make sense, please contact me because it was a headache of a test to write!

In [131]:
# Test next_availability by checking if it overlaps with any reservations or is NOT the earliest reservation.
def test_next_availability(simulations=100, debug=False):
    start_time = time.time()
    calendar = Calendars(min_start=LOWER_LIMIT, max_end=UPPER_LIMIT)
    i = 0
    while i < simulations:
        test_reservation = Reservations()
        if recursive_scheduler(calendar.reservations, calendar, test_reservation):
            calendar.add(test_reservation)
            if debug:
                print("Your reservation is valid, it seems. New calendar: ", calendar.reservations)
        elif debug:
            print("It's invalid, sorry.")
            print("Your request: ", test_reservation)
            print("Existing bookings: ", calendar.reservations)
        
        next_available_start, next_available_end = next_availability(calendar)
        
        calendar.reservations.sort(key = lambda res: res.date_ended)
        test_bookings = calendar.reservations
        while len(test_bookings) > 1:
            if test_bookings[0].date_ended == next_available_start:
                if test_bookings[1].date_started == next_available_end:
                    break
                else:
                    case = 1
                    print(f"Error Type({case}), next_availability did not propose the full range of availability.")
                    print("Real Availability Start: ", test_bookings[0].date_ended)
                    print("Real Availability End: ", test_bookings[1].date_started)
                    print(f"The availability which was proposed: {next_available_start} to {next_available_end}")
                    return None
            elif test_bookings[0].date_ended < next_available_start:
                if test_bookings[0].date_ended == test_bookings[1].date_started:
                    discard_reservations = test_bookings.pop(0)
                elif test_bookings[1].date_started >= next_available_end:
                    #NOTE: this case usually seems to be described by the fact that today's date is not equal to test_bookings[0].date_ended
                    case = 2
                    print(f"Error Type({case}), next_availability did not propose the full range of availability.")
                    print("Real Availability Start: ", test_bookings[0].date_ended)
                    print("Real Availability End: ", test_bookings[1].date_started)
                    print(f"The availability which was proposed: {next_available_start} to {next_available_end}")
                    return None
                elif test_bookings[1].date_started < next_available_start:
                    if test_bookings[1].date_ended > next_available_start:
                        case = 3
                        print(f"Error Type({case}), invalid availability provided.")
                        print(f"Availability: {next_available_start} to {next_available_end}")
                        print(f"Overlapping with - ")
                        print("Reservation Start: ", test_bookings[1].date_started)
                        print("Reservation End: ", test_bookings[1].date_ended)
                        print(f"The availability which was proposed: {next_available_start} to {next_available_end}")
                        return None
                    else:
                        case = 4
                        print(f"Error Type({case}), Looks like we missed a valid reservation between.")
                        print("Earliest Availability Start: ", test_bookings[0].date_ended)
                        print("Earliest Availability End: ", test_bookings[1].date_started)
                        print("Tailing Reservation: ", test_bookings[1])
                        print(f"The availability which was proposed: {next_available_start} to {next_available_end}")
                        return None
            elif test_bookings[0].date_start < next_available_start:
                case = 5
                print(f"Error Type({case}), invalid availability provided.")
                print(f"Availability: {next_available_start} to {next_available_end}")
                print(f"Overlapping with reservation: {test_bookings[0]}")
                return None
            else:
                break
        i += 1
    return f"Test passed. {simulations} simulations executed in {time.time() - start_time} seconds."

In [133]:
# Test conditions
SIMULATION_COUNT = 100
DEBUG = False

# test_next_availability(simulations=SIMULATION_COUNT, debug=DEBUG)

The tests for next_availability seem to fail on case 4. Still developing the solution...